In [1]:
from utils import *
from model import build_decoder_model

%load_ext autoreload
%autoreload 2

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vocab = Vocab("finished_files/vocab", VOCAB_SIZE)









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: perisic


In [3]:
model = build_decoder_model(None)
model.load_model("loss3")
torch.cuda.memory_allocated()

271219712

In [4]:
class TestBatcher():
    
    def __init__(self, vocab, filename, max_article_len, max_target_len):
        generator = example_gen(filename)

        unknown_words_cnt = 0
        self.articles = []
        self.targets = []
        self.unknown_words = []
        self.batches = []
        while True:
            
            try:
                article_text, abstract_text = next(generator)
                
                article_ids, unknown_words_list = article2ids(article_text, vocab)
                art_len = min(max_article_len, len(article_ids))
                article_ids = article_ids[:art_len]
                
                target = abstract2ids(abstract_text, vocab, unknown_words_list)
                tar_len = min(max_target_len, len(target))
                target.append(STOP_DECODING_ID)
                target = target[:tar_len]
                
                
                self.articles.append(np.array([article_ids]).T)
                self.targets.append(np.array(target))
                self.unknown_words.append(unknown_words_list)
                unknown_words_cnt = max(len(unknown_words_list), unknown_words_cnt)
                self.batches.append((np.array([article_ids]).T, np.array(target), unknown_words_list))
                #print(article_text)
            except:
                break
        print(len(self.batches))
        print(unknown_words_cnt)
    
    def generator(self):
        for batch in self.batches:
            yield batch

In [5]:
test_data = TestBatcher(vocab, "finished_files/test.bin", 400, 120)

11490
193


In [6]:
import codecs
from beam_search import run_beam_search
import os

def test_generate(model, data, beam_width, mod=0):  
    model.train(False)
    sbleu = 0
    with torch.autograd.set_grad_enabled(False):
        for i, (article_text, target, unk_words) in enumerate(data.generator()):
            #result = generate_summary(article_text, model, beam_width)
            
            '''X_batch = LongTensor(article_text)
            inputs = InputLayerState("input", True, X_batch)
            batch_size = X_batch.shape[1]
            symbols = LongTensor([[START_DECODING_ID]*batch_size])
            result = []
            model.eval_run_encoder(inputs, beam_search=False)
            for k in range(100):
                hidden = model.decode(symbols)
                symbols = hidden.argmax(-1)
                result.append(symbols.squeeze(0).item())
            '''
            result = run_beam_search(model, vocab, article_text, beam_width, 100)
            #print(result)
            #print(target)
            
            target = clean_sentence(target)
            result = clean_sentence(result)
            bleu = sentence_bleu([result], target, weights=(0.5, 0.5, 0., 0.))
            result = outputids2words(result, vocab, unk_words)
            target = outputids2words(target, vocab, unk_words)
            #print('{:.4f}'.format(bleu))
            sbleu += bleu
            '''if (bleu < 1e-3):
                print(result)
                print(0)
                print(target)'''
            #print(result.split())
            #print('target is \n')
            #print(target.split())
            if not os.access('refs' + str(mod), os.F_OK):
                os.mkdir('refs' + str(mod), 777)
            if not os.access('res' + str(mod), os.F_OK):
                os.mkdir('res' + str(mod), 777)
            with codecs.open('refs' + str(mod) + '/ref.' + str(i) +'.txt', 'w', encoding='utf8') as reference_file:
                sentences = target.split('.')
                for s in sentences:
                    reference_file.write(s.strip() + '.\n')
                
                
            with codecs.open('res' + str(mod) + '/res.' + str(i) + '.txt', 'w', encoding='utf8') as result_file:
                sentences = result.split('.')
                for s in sentences:
                    result_file.write(s.strip() + '.\n')
        print(sbleu/1000)

In [7]:
test_generate(model,test_data, beam_width=4)

c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1.3338394197303525


In [8]:
from pyrouge import Rouge155

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res0'
r.model_dir = 'E:/NIR/drag/refs0'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2019-05-17 16:29:00,764 [MainThread  ] [INFO ]  Writing summaries.
2019-05-17 16:29:00,766 [MainThread  ] [INFO ]  Processing summaries. Saving system files to C:\Users\Anatoly\AppData\Local\Temp\tmpnven7vn7\system and model files to C:\Users\Anatoly\AppData\Local\Temp\tmpnven7vn7\model.
2019-05-17 16:29:00,767 [MainThread  ] [INFO ]  Processing files in E:/NIR/drag/res0.
2019-05-17 16:29:00,778 [MainThread  ] [INFO ]  Processing res.0.txt.
2019-05-17 16:29:00,787 [MainThread  ] [INFO ]  Processing res.1.txt.
2019-05-17 16:29:00,795 [MainThread  ] [INFO ]  Processing res.10.txt.
2019-05-17 16:29:00,812 [MainThread  ] [INFO ]  Processing res.100.txt.
2019-05-17 16:29:00,820 [MainThread  ] [INFO ]  Processing res.1000.txt.
2019-05-17 16:29:00,828 [MainThread  ] [INFO ]  Processing res.10000.txt.
2019-05-17 16:29:00,834 [MainThread  ] [INFO ]  Processing res.10001.txt.
2019-05-17 16:29:00,840 [MainThread  ] [INFO ]  Processing res.10002.txt.
2019-05-17 16:29:00,845 [MainThread  ] [INFO ] 

2019-05-17 16:29:01,421 [MainThread  ] [INFO ]  Processing res.10091.txt.
2019-05-17 16:29:01,428 [MainThread  ] [INFO ]  Processing res.10092.txt.
2019-05-17 16:29:01,433 [MainThread  ] [INFO ]  Processing res.10093.txt.
2019-05-17 16:29:01,439 [MainThread  ] [INFO ]  Processing res.10094.txt.
2019-05-17 16:29:01,445 [MainThread  ] [INFO ]  Processing res.10095.txt.
2019-05-17 16:29:01,451 [MainThread  ] [INFO ]  Processing res.10096.txt.
2019-05-17 16:29:01,457 [MainThread  ] [INFO ]  Processing res.10097.txt.
2019-05-17 16:29:01,463 [MainThread  ] [INFO ]  Processing res.10098.txt.
2019-05-17 16:29:01,468 [MainThread  ] [INFO ]  Processing res.10099.txt.
2019-05-17 16:29:01,476 [MainThread  ] [INFO ]  Processing res.101.txt.
2019-05-17 16:29:01,485 [MainThread  ] [INFO ]  Processing res.1010.txt.
2019-05-17 16:29:01,493 [MainThread  ] [INFO ]  Processing res.10100.txt.
2019-05-17 16:29:01,499 [MainThread  ] [INFO ]  Processing res.10101.txt.
2019-05-17 16:29:01,505 [MainThread  ] [I

2019-05-17 16:29:02,093 [MainThread  ] [INFO ]  Processing res.10191.txt.
2019-05-17 16:29:02,099 [MainThread  ] [INFO ]  Processing res.10192.txt.
2019-05-17 16:29:02,104 [MainThread  ] [INFO ]  Processing res.10193.txt.
2019-05-17 16:29:02,110 [MainThread  ] [INFO ]  Processing res.10194.txt.
2019-05-17 16:29:02,116 [MainThread  ] [INFO ]  Processing res.10195.txt.
2019-05-17 16:29:02,122 [MainThread  ] [INFO ]  Processing res.10196.txt.
2019-05-17 16:29:02,128 [MainThread  ] [INFO ]  Processing res.10197.txt.
2019-05-17 16:29:02,134 [MainThread  ] [INFO ]  Processing res.10198.txt.
2019-05-17 16:29:02,140 [MainThread  ] [INFO ]  Processing res.10199.txt.
2019-05-17 16:29:02,147 [MainThread  ] [INFO ]  Processing res.102.txt.
2019-05-17 16:29:02,155 [MainThread  ] [INFO ]  Processing res.1020.txt.
2019-05-17 16:29:02,164 [MainThread  ] [INFO ]  Processing res.10200.txt.
2019-05-17 16:29:02,170 [MainThread  ] [INFO ]  Processing res.10201.txt.
2019-05-17 16:29:02,176 [MainThread  ] [I

2019-05-17 16:29:02,779 [MainThread  ] [INFO ]  Processing res.10291.txt.
2019-05-17 16:29:02,785 [MainThread  ] [INFO ]  Processing res.10292.txt.
2019-05-17 16:29:02,790 [MainThread  ] [INFO ]  Processing res.10293.txt.
2019-05-17 16:29:02,795 [MainThread  ] [INFO ]  Processing res.10294.txt.
2019-05-17 16:29:02,801 [MainThread  ] [INFO ]  Processing res.10295.txt.
2019-05-17 16:29:02,807 [MainThread  ] [INFO ]  Processing res.10296.txt.
2019-05-17 16:29:02,813 [MainThread  ] [INFO ]  Processing res.10297.txt.
2019-05-17 16:29:02,820 [MainThread  ] [INFO ]  Processing res.10298.txt.
2019-05-17 16:29:02,825 [MainThread  ] [INFO ]  Processing res.10299.txt.
2019-05-17 16:29:02,832 [MainThread  ] [INFO ]  Processing res.103.txt.
2019-05-17 16:29:02,840 [MainThread  ] [INFO ]  Processing res.1030.txt.
2019-05-17 16:29:02,848 [MainThread  ] [INFO ]  Processing res.10300.txt.
2019-05-17 16:29:02,854 [MainThread  ] [INFO ]  Processing res.10301.txt.
2019-05-17 16:29:02,859 [MainThread  ] [I

2019-05-17 16:29:03,434 [MainThread  ] [INFO ]  Processing res.10390.txt.
2019-05-17 16:29:03,440 [MainThread  ] [INFO ]  Processing res.10391.txt.
2019-05-17 16:29:03,448 [MainThread  ] [INFO ]  Processing res.10392.txt.
2019-05-17 16:29:03,454 [MainThread  ] [INFO ]  Processing res.10393.txt.
2019-05-17 16:29:03,459 [MainThread  ] [INFO ]  Processing res.10394.txt.
2019-05-17 16:29:03,465 [MainThread  ] [INFO ]  Processing res.10395.txt.
2019-05-17 16:29:03,472 [MainThread  ] [INFO ]  Processing res.10396.txt.
2019-05-17 16:29:03,478 [MainThread  ] [INFO ]  Processing res.10397.txt.
2019-05-17 16:29:03,483 [MainThread  ] [INFO ]  Processing res.10398.txt.
2019-05-17 16:29:03,490 [MainThread  ] [INFO ]  Processing res.10399.txt.
2019-05-17 16:29:03,495 [MainThread  ] [INFO ]  Processing res.104.txt.
2019-05-17 16:29:03,503 [MainThread  ] [INFO ]  Processing res.1040.txt.
2019-05-17 16:29:03,510 [MainThread  ] [INFO ]  Processing res.10400.txt.
2019-05-17 16:29:03,517 [MainThread  ] [I

...

2019-05-17 16:31:21,356 [MainThread  ] [INFO ]  Processing ref.9669.txt.
2019-05-17 16:31:21,361 [MainThread  ] [INFO ]  Processing ref.967.txt.
2019-05-17 16:31:21,369 [MainThread  ] [INFO ]  Processing ref.9670.txt.
2019-05-17 16:31:21,374 [MainThread  ] [INFO ]  Processing ref.9671.txt.
2019-05-17 16:31:21,380 [MainThread  ] [INFO ]  Processing ref.9672.txt.
2019-05-17 16:31:21,385 [MainThread  ] [INFO ]  Processing ref.9673.txt.
2019-05-17 16:31:21,394 [MainThread  ] [INFO ]  Processing ref.9674.txt.
2019-05-17 16:31:21,400 [MainThread  ] [INFO ]  Processing ref.9675.txt.
2019-05-17 16:31:21,406 [MainThread  ] [INFO ]  Processing ref.9676.txt.
2019-05-17 16:31:21,412 [MainThread  ] [INFO ]  Processing ref.9677.txt.
2019-05-17 16:31:21,417 [MainThread  ] [INFO ]  Processing ref.9678.txt.
2019-05-17 16:31:21,425 [MainThread  ] [INFO ]  Processing ref.9679.txt.
2019-05-17 16:31:21,430 [MainThread  ] [INFO ]  Processing ref.968.txt.
2019-05-17 16:31:21,441 [MainThread  ] [INFO ]  Proce

2019-05-17 16:31:22,024 [MainThread  ] [INFO ]  Processing ref.9770.txt.
2019-05-17 16:31:22,029 [MainThread  ] [INFO ]  Processing ref.9771.txt.
2019-05-17 16:31:22,034 [MainThread  ] [INFO ]  Processing ref.9772.txt.
2019-05-17 16:31:22,039 [MainThread  ] [INFO ]  Processing ref.9773.txt.
2019-05-17 16:31:22,045 [MainThread  ] [INFO ]  Processing ref.9774.txt.
2019-05-17 16:31:22,050 [MainThread  ] [INFO ]  Processing ref.9775.txt.
2019-05-17 16:31:22,055 [MainThread  ] [INFO ]  Processing ref.9776.txt.
2019-05-17 16:31:22,061 [MainThread  ] [INFO ]  Processing ref.9777.txt.
2019-05-17 16:31:22,066 [MainThread  ] [INFO ]  Processing ref.9778.txt.
2019-05-17 16:31:22,071 [MainThread  ] [INFO ]  Processing ref.9779.txt.
2019-05-17 16:31:22,076 [MainThread  ] [INFO ]  Processing ref.978.txt.
2019-05-17 16:31:22,083 [MainThread  ] [INFO ]  Processing ref.9780.txt.
2019-05-17 16:31:22,088 [MainThread  ] [INFO ]  Processing ref.9781.txt.
2019-05-17 16:31:22,093 [MainThread  ] [INFO ]  Proc

2019-05-17 16:31:22,672 [MainThread  ] [INFO ]  Processing ref.9872.txt.
2019-05-17 16:31:22,678 [MainThread  ] [INFO ]  Processing ref.9873.txt.
2019-05-17 16:31:22,683 [MainThread  ] [INFO ]  Processing ref.9874.txt.
2019-05-17 16:31:22,691 [MainThread  ] [INFO ]  Processing ref.9875.txt.
2019-05-17 16:31:22,696 [MainThread  ] [INFO ]  Processing ref.9876.txt.
2019-05-17 16:31:22,703 [MainThread  ] [INFO ]  Processing ref.9877.txt.
2019-05-17 16:31:22,709 [MainThread  ] [INFO ]  Processing ref.9878.txt.
2019-05-17 16:31:22,714 [MainThread  ] [INFO ]  Processing ref.9879.txt.
2019-05-17 16:31:22,720 [MainThread  ] [INFO ]  Processing ref.988.txt.
2019-05-17 16:31:22,729 [MainThread  ] [INFO ]  Processing ref.9880.txt.
2019-05-17 16:31:22,734 [MainThread  ] [INFO ]  Processing ref.9881.txt.
2019-05-17 16:31:22,740 [MainThread  ] [INFO ]  Processing ref.9882.txt.
2019-05-17 16:31:22,746 [MainThread  ] [INFO ]  Processing ref.9883.txt.
2019-05-17 16:31:22,753 [MainThread  ] [INFO ]  Proc

2019-05-17 16:31:23,322 [MainThread  ] [INFO ]  Processing ref.9974.txt.
2019-05-17 16:31:23,328 [MainThread  ] [INFO ]  Processing ref.9975.txt.
2019-05-17 16:31:23,333 [MainThread  ] [INFO ]  Processing ref.9976.txt.
2019-05-17 16:31:23,338 [MainThread  ] [INFO ]  Processing ref.9977.txt.
2019-05-17 16:31:23,344 [MainThread  ] [INFO ]  Processing ref.9978.txt.
2019-05-17 16:31:23,349 [MainThread  ] [INFO ]  Processing ref.9979.txt.
2019-05-17 16:31:23,355 [MainThread  ] [INFO ]  Processing ref.998.txt.
2019-05-17 16:31:23,363 [MainThread  ] [INFO ]  Processing ref.9980.txt.
2019-05-17 16:31:23,368 [MainThread  ] [INFO ]  Processing ref.9981.txt.
2019-05-17 16:31:23,373 [MainThread  ] [INFO ]  Processing ref.9982.txt.
2019-05-17 16:31:23,378 [MainThread  ] [INFO ]  Processing ref.9983.txt.
2019-05-17 16:31:23,383 [MainThread  ] [INFO ]  Processing ref.9984.txt.
2019-05-17 16:31:23,389 [MainThread  ] [INFO ]  Processing ref.9985.txt.
2019-05-17 16:31:23,398 [MainThread  ] [INFO ]  Proc

---------------------------------------------
1 ROUGE-1 Average_R: 0.53210 (95%-conf.int. 0.52954 - 0.53462)
1 ROUGE-1 Average_P: 0.24896 (95%-conf.int. 0.24718 - 0.25086)
1 ROUGE-1 Average_F: 0.33055 (95%-conf.int. 0.32862 - 0.33240)
---------------------------------------------
1 ROUGE-2 Average_R: 0.22798 (95%-conf.int. 0.22550 - 0.23055)
1 ROUGE-2 Average_P: 0.10601 (95%-conf.int. 0.10466 - 0.10741)
1 ROUGE-2 Average_F: 0.14091 (95%-conf.int. 0.13921 - 0.14260)
---------------------------------------------
1 ROUGE-3 Average_R: 0.12895 (95%-conf.int. 0.12661 - 0.13128)
1 ROUGE-3 Average_P: 0.06026 (95%-conf.int. 0.05904 - 0.06149)
1 ROUGE-3 Average_F: 0.07986 (95%-conf.int. 0.07834 - 0.08138)
---------------------------------------------
1 ROUGE-4 Average_R: 0.08503 (95%-conf.int. 0.08292 - 0.08710)
1 ROUGE-4 Average_P: 0.03994 (95%-conf.int. 0.03886 - 0.04106)
1 ROUGE-4 Average_F: 0.05277 (95%-conf.int. 0.05140 - 0.05416)
---------------------------------------------
1 ROUGE-L Aver

model.load_model("cov_loss2")

test_generate(model,test_data, beam_width=4, mod=1)

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res1'
r.model_dir = 'E:/NIR/drag/refs1'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.A.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

In [9]:
0.10897

0.10897